# SpaceX Launch Records Dashboard
This notebook builds a Dash app for visualizing SpaceX launch data.

In [1]:

import pandas as pd
import plotly.express as px
#Dash imports (compatible with Dash 2.x+)
try:
    from dash import Dash, html, dcc
except ImportError:
    import dash
    import dash_html_components as html
    import dash_core_components as dcc
    from dash.dependencies import Input, Output
    Dash = dash.Dash
else:
    from dash.dependencies import Input, Output

# Load dataset
spacex_data_frame = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_data_frame['Payload Mass (kg)'].max()
min_payload = spacex_data_frame['Payload Mass (kg)'].min()

# Create Dash app
app = Dash(__name__)


## App Layout

In [2]:

app.layout = html.Div([
    html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center'}),

    # TASK 1: Dropdown for Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label': 'All Sites', 'value': 'ALL'},
            {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
            {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
            {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
            {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder='Select a Launch Site',
        searchable=True
    ),
    html.Br(),

    # TASK 2: Pie chart
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),

    # TASK 3: Range slider
    dcc.RangeSlider(
        id='payload-slider',
        min=0, max=10000, step=1000,
        marks={i: str(i) for i in range(0, 10001, 2000)},
        value=[min_payload, max_payload]
    ),
    html.Br(),

    # TASK 4: Scatter plot
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])


## Callbacks for Interactive Charts

In [3]:

# Callback for Pie Chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def update_pie(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(spacex_data_frame, names='Launch Site', values='Class',
                     title='Total Successful Launches for All Sites')
    else:
        filtered_df = spacex_data_frame[spacex_data_frame['Launch Site'] == selected_site]
        fig = px.pie(filtered_df, names='Class',
                     title=f'Total Success vs Failure for site {selected_site}')
    return fig

# Callback for Scatter Chart
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def update_scatter(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_data_frame[(spacex_data_frame['Payload Mass (kg)'] >= low) & 
                                    (spacex_data_frame['Payload Mass (kg)'] <= high)]
    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]
    fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='Class',
                     color='Booster Version',
                     title='Correlation between Payload and Launch Outcome')
    return fig


## Run the App

In [4]:
if __name__ == '__main__':
    app.run(debug=True)

